#### FFT/iFFT vs Circular Convolution experiments

In [ ]:
# %pip install numpy scipy pandas plotly nbformat


In [ ]:
import numpy as np
from numpy.fft import fft, ifft
import scipy
import scipy.signal
import plotly.express as px
import plotly.graph_objects as go

from circular_convolution import cc_direct, cc_fft


In [ ]:
N = 128
x = np.linspace(0, 50, N)

true_y = np.sin(x)
y_range = np.max(true_y) - np.min(true_y)
y = true_y + np.random.normal(scale=y_range/10, size=N) + np.sin(x*1.33)

fig = px.line(y=y)
# fig.add_scatter(y=scipy.signal.savgol_filter(y, 20, 5), name='smooth')
fig.show()


In [ ]:
kernel = np.concat([np.zeros((N//4, )), np.ones((N//2,)), np.zeros((N//4,))])
# kernel = np.random.normal(scale=1, size=N)

y_direct = cc_direct(y, kernel)
y_fft = cc_fft(y, kernel)
fig = go.Figure()
fig.add_scatter(y=y_direct)
fig.add_scatter(y=y_fft)
fig.show()

print('Standard deviation:', np.var(y_direct - y_fft))


#### Fixed point math convertion experiments

In [ ]:
import importlib
from pathlib import Path
import sys

def import_from_path(module_name, file_path):
    """Import a module given its name and file path."""
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module


module_name = 'test'
module_path = Path('../sim/cocotb/test.py').resolve()
cotest = import_from_path(module_name, module_path)
cotest

In [ ]:
importlib.reload(cotest)
import numpy as np

x = np.linspace(0, 10)
y = np.sin(x)


In [ ]:
import itertools
import fxpmath

bin_array = cotest.convert_float_to_logic(y)

print(np.asarray([fxpmath.Fxp('b'+''.join(binstr), signed=True, n_word=16, n_frac=12) for binstr in itertools.batched(bin_array.binstr, cotest.XLEN)]))
cotest.convert_logic_to_float(cotest.convert_float_to_logic(x))

In [ ]:
arr = np.asarray(list(map(float, '''0.26489258  1.0859375  -0.25854492 -0.30517578  0.7421875   0.64379883
 -1.85620117 -1.4699707   0.56030273  1.83496094  1.88085938 -0.15527344
  0.2722168  -1.65917969  0.82666016  1.40722656  1.76806641 -0.55200195
  0.81811523 -0.54296875  1.53149414 -0.03515625 -0.03540039  0.47875977
  1.83935547 -0.07250977 -0.45654297  0.3425293   1.18164062  0.68334961
 -2.05786133 -0.77246094 -0.3684082   0.15258789 -1.51806641  0.21020508
 -0.8203125  -1.3371582   0.22143555  0.50341797  0.4152832  -0.41381836
 -0.90771484 -0.88012695  1.17260742 -0.65356445  0.81958008 -0.45654297
 -0.07666016  0.66967773 -2.16186523 -0.0949707   0.64306641  0.32055664
 -1.39404297 -0.73339844  0.72583008  1.70874023  1.57885742  1.47900391
 -1.81030273 -0.99462891 -1.23388672 -2.58789062 -1.46484375 -0.03808594
  0.11767578  1.18896484  0.92480469 -0.0546875  -1.34082031 -2.46337891
  1.19970703  0.9296875   1.43945312  1.66162109 -1.41967773 -0.0949707
  0.62280273 -1.52099609 -0.7668457  -0.32055664  1.14453125  0.61450195
  0.17260742  0.14477539 -1.34716797  1.41723633  1.53564453 -0.01098633
 -0.33911133  1.53564453  0.1640625   0.9362793  -1.30297852 -1.1171875
 -0.58740234 -0.63061523  0.39990234  0.93701172 -1.02441406 -0.64233398
  0.66113281 -0.89477539 -1.43261719  0.09960938  0.79760742 -0.69775391
  0.9543457  -2.21557617 -0.77368164  0.70336914 -0.27172852  1.26293945
 -0.47436523  0.36645508 -0.76660156 -1.59033203  0.44384766  1.03540039
 -0.05078125  1.18139648 -0.27636719 -0.69238281 -0.9609375   0.33032227
  0.40454102  1.64111328'''.split())))
np.real(np.fft.ifft(np.fft.fft(arr)*np.fft.fft(np.ones(128))))

In [ ]:
received = np.asarray(list(map(float, '''0.61328125 -1.51171875 -5.7265625   4.9609375   3.9296875  -3.265625
  1.87109375  7.5390625  -4.0078125   1.08203125  1.125      -0.2890625
 -1.3046875  -6.6171875  -5.20703125 -7.7265625  -6.328125   -6.9375
  7.0390625   3.33203125  6.3984375  -0.04296875 -5.03515625 -0.63671875
 -6.671875    2.37890625  0.96875    -6.80859375  3.71875    -7.984375
  2.94921875  2.4921875  -4.671875   -2.08203125 -2.27734375  4.46875
 -0.41796875  2.0234375   5.3359375   4.87109375  6.140625    3.3046875
  2.7578125  -6.73046875  2.1640625  -0.90625     5.984375   -0.6171875
  5.98828125 -5.34765625  4.96484375  5.453125   -0.05078125  0.55078125
  3.5703125   2.5859375   7.32421875 -0.2265625  -2.91796875 -4.17578125
 -2.68359375  0.453125    7.90625     6.84765625 -1.61328125 -0.98828125
 -0.94140625  1.8046875   7.4296875   2.75390625 -7.484375    2.6796875
  7.24609375 -4.171875   -7.33984375  2.23046875  1.2734375   7.34375
  7.33984375  4.66796875  0.65234375 -1.53125    -0.09765625  2.37109375
  5.97265625  7.19921875 -7.0390625  -5.953125    2.1640625   6.1953125
 -0.8046875  -4.98046875  4.38671875  4.3984375  -7.55859375  4.3046875
  3.71875     4.3046875  -0.8359375   7.31640625 -4.26171875 -0.39453125
 -7.37890625 -2.015625   -6.31640625  1.5         4.01953125 -0.62109375
  5.44140625  7.56640625  3.859375   -3.59375     3.15234375 -5.32421875
 -5.171875   -3.46875     2.06640625  1.31640625 -1.93359375  1.7890625
 -2.59375    -5.40234375  3.74609375 -3.2890625   2.96484375  7.890625
 -2.265625    0.9921875'''.split())))
px.line(received)